In [1]:
import sys
# setting path
sys.path.append("/content/drive/MyDrive/simclr")
sys.path.append("/content/drive/MyDrive/simclr/layers")
sys.path.append("/content/drive/MyDrive/simclr/models")
sys.path.append("/content/drive/MyDrive/simclr/loss")
sys.path.append("/content/drive/MyDrive/simclr/optim")
sys.path.append("/content/drive/MyDrive/simclr/dataloader")
sys.path.append("/content/drive/MyDrive/simclr/instance-discrim")
sys.path.append("/content/drive/MyDrive/simclr/dloader")

In [2]:
from simclr import SimClr
from ntxent import nt_xent_loss
from ntxentgit import SimCLR_Loss
from augment import TransformsSimCLR
from utils import *
from LARS import LARS
from downstream import DownStream
from dataloader import *
from ResNetCifar import ResNetCifar
from utilsInstance import *
from NCEAverage import *
from NCECriterion import *
from alias_multinomial import *
from Cifar10Instance import *
from downstreamnew import DownStreamNew

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pickle

In [4]:
shuffle = True
train_loader_ds, val_loader_ds = train_loader_dstream("CIFAR10",256,0.1,shuffle)

Files already downloaded and verified


In [5]:
model = SimClr('resnet50',128).cuda()
model.load_state_dict(torch.load("/content/drive/MyDrive/simclr/saved-models/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM"))

<All keys matched successfully>

In [6]:
model_ds = DownStreamNew(model.encoder,model.projector.layers[:2],2048,10).cuda()
optimizer = torch.optim.Adam(model_ds.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
epochs = 100
model_ds,train_loss_list,train_acc_list,val_loss_list,val_acc_list = train_ds(train_loader_ds, val_loader_ds, model_ds, criterion, optimizer,epochs,True,path="/content/drive/MyDrive/simclr/dstream-saved-models/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM")

Epoch: 0, step: 50/175, training_loss: 0.8006798028945923, training_acc: 0.734375
Epoch: 0, step: 100/175, training_loss: 0.5786304473876953, training_acc: 0.80859375
Epoch: 0, step: 150/175, training_loss: 0.6826297044754028, training_acc: 0.7734375
Epoch: 0 completed, average train loss: 0.9237893676757812, average training_acc: 0.7155133928571429, average validation loss: 0.7000172953856619, average validation_acc: 0.7514391447368421 time taken: 0.42774219115575157 mins
Epoch: 1, step: 50/175, training_loss: 0.6172121167182922, training_acc: 0.75390625
Epoch: 1, step: 100/175, training_loss: 0.6079005599021912, training_acc: 0.796875
Epoch: 1, step: 150/175, training_loss: 0.6296188235282898, training_acc: 0.796875
Epoch: 1 completed, average train loss: 0.6875314293588911, average training_acc: 0.75765625, average validation loss: 0.676667006392228, average validation_acc: 0.7633634868421053 time taken: 0.4261431097984314 mins
Epoch: 2, step: 50/175, training_loss: 0.61876171827316

In [7]:
with open("/content/drive/MyDrive/simclr/dstream-trainloss-logs/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM", "wb") as fp:   #Pickling
    pickle.dump(train_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-trainacc-logs/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM", "wb") as fp:   #Pickling
    pickle.dump(train_acc_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valloss-logs/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM", "wb") as fp:   #Pickling
    pickle.dump(val_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valacc-logs/CIFAR10-RES50-SIMCLR-BS256-PD128-ADAM", "wb") as fp:   #Pickling
    pickle.dump(val_acc_list, fp)

In [8]:
test_loader = test_loader("CIFAR10",128)

Files already downloaded and verified


In [9]:
test_ds(model_ds,test_loader)

Test accuracy: 0.7828525641025641, time taken: 0.08238502740859985
